In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
import cv2
import os

In [ ]:
!pip freeze > requirnments.txt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load text data
text_data = pd.read_csv('/content/drive/MyDrive/EmotionDetection/iemocap_full_dataset.csv')

# Function to preprocess image
def preprocess_image(image):
    # Resize image to a fixed size (e.g., 48x48 pixels)
    resized_image = cv2.resize(image, (48, 48))
    # Convert image to grayscale
    grayscale_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    # Normalize pixel values to [0, 1]
    normalized_image = grayscale_image / 255.0
    return normalized_image

In [ ]:
# Define the directory containing CK+ images
ck_plus_images_dir = '/content/drive/MyDrive/EmotionDetection/CK+'

# Initialize lists to store images and labels
images = []
labels = []

# Define a mapping of emotion labels to numerical values
emotion_label_map = {
    'anger': 0,
    'contempt': 1,
    'disgust': 2,
    'fear': 3,
    'happy': 4,
    'sadness': 5,
    'surprise': 6
}

In [ ]:
# Initialize an empty list to store images
images = []

labels_list = []

# Loop through each directory (corresponding to each emotion label)
for emotion_dir in os.listdir(ck_plus_images_dir):
    # Skip non-directory files
    if not os.path.isdir(os.path.join(ck_plus_images_dir, emotion_dir)):
        continue

    # Map the emotion label to its numerical value
    label = emotion_label_map.get(emotion_dir.lower())  # Use lower() to ensure case-insensitive mapping
    if label is None:
        continue  # Skip if the emotion label is not defined in the mapping

    # Loop through each image file in the directory
    for filename in os.listdir(os.path.join(ck_plus_images_dir, emotion_dir)):
        if filename.endswith('.png'):  # Assuming images are in PNG format
            # Read the image
            image = cv2.imread(os.path.join(ck_plus_images_dir, emotion_dir, filename))
            if image is None:
                continue  # Skip if image loading fails
            # Preprocess the image
            preprocessed_image = preprocess_image(image)
            # Append the preprocessed image to the list of images
            images.append(preprocessed_image)
            # Append the label to the list of labels
            labels_list.append(label)

# Convert the list of labels to a NumPy array
labels = np.array(labels_list)


In [ ]:
# Convert lists to NumPy arrays
images = np.array(images)
labels = np.array(labels)

In [ ]:
import numpy as np

# Check the label distribution
unique_labels, label_counts = np.unique(labels, return_counts=True)
print("Unique Labels:", unique_labels)
print("Label Counts:", label_counts)


Unique Labels: [0 1 2 3 4 5 6]
Label Counts: [135  54 177  75 207  84 249]


In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


# Initialize and train a Support Vector Machine (SVM) classifier
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train.reshape(len(X_train), -1), y_train)

# Initialize and train a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100)
rf_classifier.fit(X_train.reshape(len(X_train), -1), y_train)

RandomForestClassifier()

In [ ]:
# Apply K-Means clustering
kmeans = KMeans(n_clusters=3)
clusters = kmeans.fit_predict(X_train.reshape(len(X_train), -1))

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
# Define the CNN model for facial expression analysis
def build_cnn_model():
    model = tf.keras.Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(7, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Build the CNN model
cnn_model = build_cnn_model()
cnn_model.fit(X_train.reshape(-1, 48, 48, 1), y_train, epochs=10)

Epoch 1/10
25/25 [==============================] - 4s 17ms/step - loss: 1.7774 - accuracy: 0.2959
Epoch 2/10
25/25 [==============================] - 0s 5ms/step - loss: 1.4080 - accuracy: 0.5319
Epoch 3/10
25/25 [==============================] - 0s 5ms/step - loss: 0.9368 - accuracy: 0.6837
Epoch 4/10
25/25 [==============================] - 0s 5ms/step - loss: 0.5899 - accuracy: 0.8061
Epoch 5/10
25/25 [==============================] - 0s 5ms/step - loss: 0.3465 - accuracy: 0.9069
Epoch 6/10
25/25 [==============================] - 0s 5ms/step - loss: 0.2132 - accuracy: 0.9349
Epoch 7/10
25/25 [==============================] - 0s 5ms/step - loss: 0.1610 - accuracy: 0.9592
Epoch 8/10
25/25 [==============================] - 0s 5ms/step - loss: 0.1088 - accuracy: 0.9732
Epoch 9/10
25/25 [==============================] - 0s 5ms/step - loss: 0.0880 - accuracy: 0.9796
Epoch 10/10
25/25 [==============================] - 0s 5ms/step - loss: 0.0639 - accuracy: 0.9872


In [ ]:
# Evaluate SVM model
svm_accuracy = svm_classifier.score(X_test.reshape(len(X_test), -1), y_test)

# Evaluate Random Forest model
rf_accuracy = rf_classifier.score(X_test.reshape(len(X_test), -1), y_test)

print("SVM Accuracy:", svm_accuracy)
print("Random Forest Accuracy:", rf_accuracy)

# Evaluate CNN model
cnn_loss, cnn_accuracy = cnn_model.evaluate(X_test.reshape(-1, 48, 48, 1), y_test)

print("CNN Accuracy:", cnn_accuracy)

SVM Accuracy: 0.9949238578680203
Random Forest Accuracy: 0.9847715736040609
7/7 [==============================] - 0s 20ms/step - loss: 0.0835 - accuracy: 0.9746
CNN Accuracy: 0.9746192693710327


In [ ]:
# Save the pre-trained CNN model
cnn_model.save('emotion_recognition_model.keras')

# Load the pretrained CNN model for emotion detection
cnn_model = tf.keras.models.load_model('emotion_recognition_model.keras')

# Define the emotion labels
emotion_labels = {0: 'Anger', 1: 'Contempt', 2: 'Disgust', 3: 'Fear', 4: 'Happy', 5: 'Sadness', 6: 'Surprise'}

# Function to detect emotion from an image frame
def detect_emotion(frame):
    # Preprocess the frame
    resized_frame = cv2.resize(frame, (48, 48))
    grayscale_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)
    normalized_frame = grayscale_frame / 255.0
    input_frame = np.expand_dims(normalized_frame, axis=-1)
    input_frame = np.expand_dims(input_frame, axis=0)  # Add batch dimension

     # Perform emotion prediction
    predictions = cnn_model.predict(input_frame)
    predicted_class = np.argmax(predictions)
    emotion_label = emotion_labels[predicted_class]

    return emotion_label


In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Assuming the model is saved as 'emotion_recognition_model.keras' in your Drive
cnn_model = tf.keras.models.load_model('/content/emotion_recognition_model.keras')
print("Model loaded successfully!")


# Define the emotion labels
emotion_labels = {0: 'Anger', 1: 'Contempt', 2: 'Disgust', 3: 'Fear', 4: 'Happy', 5: 'Sadness', 6: 'Surprise'}

image_path = "/content/drive/MyDrive/EmotionDetection/S029_001_00000019.png"
image = cv2.imread(image_path)
print('ok')


Model loaded successfully!
ok


In [ ]:
# Preprocess the image
def preprocess_image(image):
  # Resize image to a fixed size (e.g., 48x48 pixels) following the model's input shape
  resized_image = cv2.resize(image, (48, 48))
  # Convert image to grayscale
  grayscale_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
  # Normalize pixel values to [0, 1]
  normalized_image = grayscale_image / 255.0
  return normalized_image

In [ ]:
# Preprocess the image using the defined function
preprocessed_image = preprocess_image(image)

# Add a batch dimension for the CNN model
input_frame = np.expand_dims(preprocessed_image, axis=-1)
input_frame = np.expand_dims(input_frame, axis=0)

# Now you have the preprocessed image (`input_frame`) ready for prediction
# using the loaded CNN model (`cnn_model`)

# Example usage (assuming you have a loaded `cnn_model`)
predictions = cnn_model.predict(input_frame)
predicted_class = np.argmax(predictions)
emotion_label = emotion_labels[predicted_class]

print("Predicted Emotion:", emotion_label)

1/1 [==============================] - 0s 264ms/step
Predicted Emotion: Anger


In [ ]:
! pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.5 MB/s eta 0:00:00


In [4]:
%%writefile app.py
import streamlit as st
import cv2
import numpy as np
import tensorflow as tf

# Load the pre-trained CNN model for emotion detection
cnn_model = tf.keras.models.load_model('emotion_recognition_model.keras')

# Define the emotion labels
emotion_labels = {0: 'Anger', 1: 'Contempt', 2: 'Disgust', 3: 'Fear', 4: 'Happy', 5: 'Sadness', 6: 'Surprise'}

# Function to preprocess image
def preprocess_image(image):
    resized_image = cv2.resize(image, (48, 48))
    grayscale_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    normalized_image = grayscale_image / 255.0
    return normalized_image

# Function to detect emotion from an image frame
def detect_emotion(frame):
    resized_frame = cv2.resize(frame, (48, 48))
    grayscale_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)
    normalized_frame = grayscale_frame / 255.0
    input_frame = np.expand_dims(normalized_frame, axis=-1)
    input_frame = np.expand_dims(input_frame, axis=0)  # Add batch dimension

    predictions = cnn_model.predict(input_frame)
    predicted_class = np.argmax(predictions)
    emotion_label = emotion_labels[predicted_class]

    return emotion_label

# Streamlit app
def main():
    st.title("Emotion Detection App")

    uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "png"])

    if uploaded_file is not None:
        try:
            # Read the image
            image = cv2.imdecode(np.fromstring(uploaded_file.read(), np.uint8), 1)

            # Display the uploaded image
            st.image(image, caption='Uploaded Image', use_column_width=True)

            # Detect emotion upon button press
            if st.button('Predict Emotion'):
                # Preprocess the image
                preprocessed_image = preprocess_image(image)

                # Add a batch dimension
                input_frame = np.expand_dims(preprocessed_image, axis=-1)
                input_frame = np.expand_dims(input_frame, axis=0)

                # Predict emotion
                predicted_emotion = detect_emotion(image)

                # Display predicted emotion
                st.success(f"Predicted Emotion: {predicted_emotion}")
        except Exception as e:
            st.error(f"Error: {e}")

if __name__ == '__main__':
    main()


Writing app.py


In [5]:
!wget -q -O - ipv4.icanhazip.com

34.139.9.79


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501

/bin/bash: line 1: streamlit: command not found
npx: installed 22 in 3.765s
your url is: https://thick-clubs-wash.loca.lt
